In [7]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

# PostgreSQL connection details
conn = psycopg2.connect(
    dbname="sw_warehouse",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Drop all tables if they exist
drop_tables_query = """
    DROP TABLE IF EXISTS scarers, monsters, door_history, doors, timezone_locations CASCADE;
"""

try:
    # Execute the drop tables query
    cur.execute(drop_tables_query)
    conn.commit()
    print("All tables dropped successfully!")
except Exception as e:
    print(f"Error dropping tables: {e}")
    conn.rollback()


All tables dropped successfully!
All tables dropped successfully!


In [8]:
# Setup Faker
fake = Faker()

# Create table
cur.execute("""
    CREATE TABLE monsters (
        monster_id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        scare_level INT,
        department VARCHAR(50)
    );
""")

# Insert random data
departments = ['Scare Floor A', 'Scare Floor B', 'R&D', 'Administration']

for _ in range(10):
    name = fake.first_name()
    scare_level = random.randint(1, 10)
    department = random.choice(departments)
    
    cur.execute(
        "INSERT INTO monsters (name, scare_level, department) VALUES (%s, %s, %s)",
        (name, scare_level, department)
    )

# Commit and close
conn.commit()

print("Table 'monsters' created and populated with random data!")


Table 'monsters' created and populated with random data!
Table 'monsters' created and populated with random data!


In [9]:
# Create scarers table
cur.execute("""
    CREATE TABLE IF NOT EXISTS scarers (
        scarer_id SERIAL PRIMARY KEY,
        monster_id INT,
        assigned_floor VARCHAR(50),
        active BOOLEAN,
        FOREIGN KEY (monster_id) REFERENCES monsters(monster_id)
    );
""")

# Fetch monster IDs from the monsters table
cur.execute("SELECT monster_id FROM monsters;")
monster_ids = [row[0] for row in cur.fetchall()]

# Randomly select a subset to be scarers
num_scarers = random.randint(int(len(monster_ids) * 0.4), int(len(monster_ids) * 0.8))
scarer_monsters = random.sample(monster_ids, num_scarers)

# Insert randomized scarers
for monster_id in scarer_monsters:
    assigned_floor = random.choice(['Scare Floor A', 'Scare Floor B', 'Scare Floor C', 'Overflow'])
    active = random.choice([True, False, True, True])  # Higher chance of active
    cur.execute("""
        INSERT INTO scarers (monster_id, assigned_floor, active)
        VALUES (%s, %s, %s);
    """, (monster_id, assigned_floor, active))

conn.commit()

print(f"{len(scarer_monsters)} scarers inserted successfully!")

4 scarers inserted successfully!
4 scarers inserted successfully!


In [10]:
try:
    # Create the doors table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS doors (
        door_id SERIAL PRIMARY KEY,
        door_serial_number VARCHAR(50) UNIQUE NOT NULL,
        door_history_id INT NOT NULL,
        warehouse_location_id INT NOT NULL,
        timezone_location_id INT NOT NULL,
        status VARCHAR(20) NOT NULL,
        last_service_date DATE,
        times_used INT DEFAULT 0,
        total_energy_output NUMERIC(10, 2) DEFAULT 0.00,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'doors' created successfully!")

except Exception as e:
    conn.rollback()
    print(f"Error creating table: {e}")

# Generate random door data
def generate_door_data(num_records=50):
    door_data = []
    for _ in range(num_records):
        door_serial = fake.unique.bothify(text='DOOR-####-????')
        door_history_id = random.randint(1, 10)
        warehouse_location_id = random.randint(1, 10)
        timezone_location_id = random.randint(1, 10)
        status = random.choice(['available', 'in-use', 'maintenance', 'retired'])
        last_service_date = fake.date_between(start_date='-2y', end_date='today')
        times_used = random.randint(0, 500)
        total_energy_output = round(random.uniform(0, 10000), 2)
        created_at = fake.date_time_between(start_date='-3y', end_date='now')

        door_data.append((
            door_serial, door_history_id, warehouse_location_id, timezone_location_id,
            status, last_service_date, times_used, total_energy_output, created_at
        ))
    return door_data

# Insert data safely
def insert_doors(data):
    try:
        insert_query = """
            INSERT INTO doors (
                door_serial_number, door_history_id, warehouse_location_id, timezone_location_id,
                status, last_service_date, times_used, total_energy_output, created_at
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} doors inserted successfully!")
    except Exception as e:
        conn.rollback()
        print(f"Error inserting data: {e}")

# Run the full process
door_records = generate_door_data(50)
insert_doors(door_records)


Table 'doors' created successfully!
50 doors inserted successfully!
Table 'doors' created successfully!
50 doors inserted successfully!


In [11]:
# Initialize Faker
fake = Faker()

# Create door_history table with foreign key constraint
try:
    create_table_query = """
    CREATE TABLE IF NOT EXISTS door_history (
        history_id SERIAL PRIMARY KEY,
        door_serial_number VARCHAR(50),
        event_type VARCHAR(50),
        event_timestamp TIMESTAMP,
        notes TEXT,
        CONSTRAINT fk_door_serial
            FOREIGN KEY(door_serial_number) 
            REFERENCES doors(door_serial_number)
            ON DELETE CASCADE
    );
    """
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'door_history' created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")
    conn.rollback()

# Fetch existing door serials from the doors table
def fetch_existing_door_serials():
    cur.execute("SELECT door_serial_number FROM doors;")
    return [row[0] for row in cur.fetchall()]

# Generate random door history data linked to existing doors
def generate_door_history_data(num_records=100):
    existing_serials = fetch_existing_door_serials()
    if not existing_serials:
        print("No doors found! Populate the doors table first.")
        return []

    history_data = []
    event_types = ['installed', 'serviced', 'repaired', 'retired']

    for _ in range(num_records):
        door_serial_number = random.choice(existing_serials)
        event_type = random.choice(event_types)
        event_timestamp = fake.date_time_between(start_date='-5y', end_date='now')
        notes = fake.sentence(nb_words=12)

        history_data.append((door_serial_number, event_type, event_timestamp, notes))

    return history_data

# Insert into the door_history table
def insert_door_history(data):
    insert_query = """
        INSERT INTO door_history (
            door_serial_number, event_type, event_timestamp, notes
        ) VALUES (%s, %s, %s, %s);
    """
    try:
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} door history records inserted successfully!")
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()

# Generate and insert 100 door history records
history_records = generate_door_history_data(100)
if history_records:
    insert_door_history(history_records)


Table 'door_history' created successfully.
100 door history records inserted successfully!
Table 'door_history' created successfully.
100 door history records inserted successfully!


In [12]:
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Create timezone_locations table
try:
    create_table_query = """
    CREATE TABLE IF NOT EXISTS timezone_locations (
        timezone_id SERIAL PRIMARY KEY,
        timezone_name VARCHAR(100),
        utc_offset INTERVAL,
        region_description TEXT
    );
    """
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'timezone_locations' created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")
    conn.rollback()

# Generate random timezone location data
def generate_timezone_locations():
    # Fixed list of common timezones for realism
    timezone_data = [
        ('Pacific Time', '-08:00', 'U.S. West Coast & parts of Canada'),
        ('Mountain Time', '-07:00', 'Rocky Mountains & interior U.S. states'),
        ('Central Time', '-06:00', 'U.S. Central Plains & Gulf states'),
        ('Eastern Time', '-05:00', 'U.S. East Coast & parts of Canada'),
        ('Atlantic Time', '-04:00', 'Canadian Maritimes'),
        ('GMT', '+00:00', 'United Kingdom and Western Africa'),
        ('Central European Time', '+01:00', 'Europe including France, Germany'),
        ('Eastern European Time', '+02:00', 'Eastern Europe including Ukraine, Greece'),
        ('Japan Standard Time', '+09:00', 'Japan'),
        ('Australian Eastern Time', '+10:00', 'Australia East Coast')
    ]

    return [(name, offset, desc) for name, offset, desc in timezone_data]

# Insert timezone location data
def insert_timezone_locations(data):
    insert_query = """
        INSERT INTO timezone_locations (
            timezone_name, utc_offset, region_description
        ) VALUES (%s, %s, %s);
    """
    try:
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} timezone locations inserted successfully!")
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()

# Generate and insert timezone data
timezone_records = generate_timezone_locations()
insert_timezone_locations(timezone_records)


Table 'timezone_locations' created successfully.
10 timezone locations inserted successfully!
Table 'timezone_locations' created successfully.
10 timezone locations inserted successfully!


In [ ]:
# Create scare_sessions table
try:
    create_table_query = """
    CREATE TABLE IF NOT EXISTS scare_sessions (
        session_id SERIAL PRIMARY KEY,
        scare_date DATE NOT NULL,
        session_number VARCHAR(20),
        door_serial_number VARCHAR(50),
        door_arrival_time TIMESTAMP,
        door_activation_time TIMESTAMP,
        door_deactivation_time TIMESTAMP,
        door_departure_time TIMESTAMP,
        scare_energy_collected NUMERIC(10, 2),
        scarer_id INT,
        assistant_id INT,
        successful_scare BOOLEAN,
        error_messages TEXT,
        notes TEXT
    );
    """
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'scare_sessions' created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")
    conn.rollback()

# Generate random scare session data with "2319!" logic
def generate_scare_sessions_data(num_records=100):
    session_data = []
    for _ in range(num_records):
        scare_date = fake.date_between(start_date='-2y', end_date='today')
        session_number = f"{random.randint(10000, 99999)}-{random.randint(1, 5)}"
        
        door_arrival_time = fake.date_time_between_dates(
            datetime_start=datetime.combine(scare_date, datetime.min.time()),
            datetime_end=datetime.combine(scare_date, datetime.max.time())
        )
        door_activation_time = door_arrival_time + timedelta(minutes=random.randint(1, 5))
        door_deactivation_time = door_activation_time + timedelta(minutes=random.randint(5, 30))
        door_departure_time = door_deactivation_time + timedelta(minutes=random.randint(1, 5))

        scarer_id = random.randint(1, 20)  # Replace with real IDs later
        assistant_id = random.randint(1, 20)  # Replace with real IDs later
        door_serial_number = fake.unique.bothify(text='DOOR-####-????')

        error_messages = None
        notes = fake.sentence(nb_words=8)
        successful_scare = True

        if random.random() < 0.2:  # 20% chance of an error
            if random.random() < 0.5:  # 50% of errors are 2319s
                error_messages = "2319! Contamination protocol initiated. Door quarantined."
                door_deactivation_time = door_activation_time + timedelta(minutes=random.randint(1, 3))
                door_departure_time = door_deactivation_time + timedelta(minutes=1)
                scare_energy_collected = round(random.uniform(0, 100), 2)
                successful_scare = False
            else:
                error_messages = random.choice([
                    "Door power fluctuation.",
                    "Child alert triggered.",
                    "Assistant malfunction.",
                    None
                ])
                scare_energy_collected = round(random.uniform(50, 300), 2)
                successful_scare = False
        else:
            scare_energy_collected = round(random.uniform(100, 1000), 2)

        session_data.append((
            scare_date, session_number, door_serial_number,
            door_arrival_time, door_activation_time, door_deactivation_time, door_departure_time,
            scare_energy_collected, scarer_id, assistant_id, successful_scare, error_messages, notes
        ))

    return session_data

# Insert into the scare_sessions table
def insert_scare_sessions(data):
    insert_query = """
        INSERT INTO scare_sessions (
            scare_date, session_number, door_serial_number,
            door_arrival_time, door_activation_time, door_deactivation_time, door_departure_time,
            scare_energy_collected, scarer_id, assistant_id, successful_scare, error_messages, notes
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    try:
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} scare session records inserted successfully!")
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()

# Generate and insert 100 scare session records
session_records = generate_scare_sessions_data(100)
insert_scare_sessions(session_records)


In [ ]:
# Create scare_team_assignments table
try:
    create_table_query = """
    CREATE TABLE IF NOT EXISTS scare_team_assignments (
        assignment_id SERIAL PRIMARY KEY,
        session_id INT NOT NULL,
        scarer_id INT NOT NULL,
        assistant_id INT NOT NULL,
        FOREIGN KEY (session_id) REFERENCES scare_sessions(session_id),
        FOREIGN KEY (scarer_id) REFERENCES scarers(scarer_id),
        FOREIGN KEY (assistant_id) REFERENCES monsters(monster_id)
    );
    """
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'scare_team_assignments' created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")
    conn.rollback()

# Generate random scare team assignment data
def generate_scare_team_assignments(num_records=100):
    assignments = []
    for session_id in range(1, num_records + 1):
        scarer_id = random.randint(1, 20)     # Example: assuming 20 scarers
        assistant_id = random.randint(1, 50)  # Example: assuming 50 monsters (assistants)
        assignments.append((session_id, scarer_id, assistant_id))
    return assignments

# Insert into the scare_team_assignments table
def insert_scare_team_assignments(data):
    insert_query = """
        INSERT INTO scare_team_assignments (
            session_id, scarer_id, assistant_id
        ) VALUES (%s, %s, %s);
    """
    try:
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} scare team assignments inserted successfully!")
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()

# Generate and insert 100 scare team assignments
assignment_records = generate_scare_team_assignments(100)
insert_scare_team_assignments(assignment_records)
